# Language Modeling 🤝 LSTM

Изучим мощь рекуррентных сетей в лице LSTM на примере решения задачи LM

# Language Modeling

Давайте введём понятия, которые нам сегодня пригодятся

Сами по себе языковые модели предназначены для того, чтобы как-то оценивать вероятность некоторой языковой конструкции. Например, мы можем свести к вероятностной модели вероятность последовательности токенов следующим образом:

$$p(x) = \prod_{i=1}^{N}p(x_i|x_1, ..., x_{i-1}) = p(x_2 | x_1) * p(x_3 | x1, x2) * ...$$
$$\log p(x) = \sum_{i=1}^{N}\log p(x_i|x_1, ..., x_{i-1})$$ 

Если мы обучим нашу модель предсказывать эти вероятности, то сможем не только предсказывать вероятность последовательности, но и получать вероятности следующих токенов. Последнее - это не что иное, как генерация текста.

# Average ArXiv Enjoyer

На прошлом семинаре мы научились правильно читать статьи, а теперь давайте научимся их писать

Писать статьи самостоятельно - это определённо прошлый век. Мы можем воспользоваться своими знаниями о машинке и автоматизировать этот процесс.

План следующий:

1. Обучим языковую модель на корпусе из статей с arXiv
2. Насемплируем несколько статей
3. Получим мировую известность и уважение в научном сообществе

In [1]:
import torch

from tqdm.auto import tqdm

## Подготовка данных

Для обучения возьмём [корпус статей с arXiv](https://www.kaggle.com/neelshah18/arxivdataset/):

In [2]:
!wget -O arXiv.zip "https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1m78dRD6OIMP4oJL4VUujXV6MVavpb3A_"

!unzip arXiv.zip
!rm arXiv.zip

--2022-03-17 14:16:53--  https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1m78dRD6OIMP4oJL4VUujXV6MVavpb3A_
Resolving drive.google.com (drive.google.com)... 108.177.121.139, 108.177.121.138, 108.177.121.101, ...
Connecting to drive.google.com (drive.google.com)|108.177.121.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-0o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/5dk5u8v5v60qrj469i30cu56nuit75v1/1647526575000/06006207853926179639/*/1m78dRD6OIMP4oJL4VUujXV6MVavpb3A_?e=download [following]
--2022-03-17 14:16:55--  https://doc-10-0o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/5dk5u8v5v60qrj469i30cu56nuit75v1/1647526575000/06006207853926179639/*/1m78dRD6OIMP4oJL4VUujXV6MVavpb3A_?e=download
Resolving doc-10-0o-docs.googleusercontent.com (doc-10-0o-docs.googleusercontent.com)... 142.250.125.132, 2607:f8b0:4001:c2f::84
Connecting to doc-10-0o-docs.googleuserc

In [3]:
import pandas as pd

arXiv_data = pd.read_json("arxivData.json")
arXiv_data.head()

,author,day,id,link,month,summary,tag,title,year
0,"[{'name': 'Ahmed Osman'}, {'name': 'Wojciech S...",1,1802.00209v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,We propose an architecture for VQA which utili...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",Dual Recurrent Attention Units for Visual Ques...,2018
1,"[{'name': 'Ji Young Lee'}, {'name': 'Franck De...",12,1603.03827v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",3,Recent approaches based on artificial neural n...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Sequential Short-Text Classification with Recu...,2016
2,"[{'name': 'Iulian Vlad Serban'}, {'name': 'Tim...",2,1606.00776v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",6,We introduce the multiresolution recurrent neu...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Multiresolution Recurrent Neural Networks: An ...,2016
3,"[{'name': 'Sebastian Ruder'}, {'name': 'Joachi...",23,1705.08142v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",5,Multi-task learning is motivated by the observ...,"[{'term': 'stat.ML', 'scheme': 'http://arxiv.o...",Learning what to share between loosely related...,2017
4,"[{'name': 'Iulian V. Serban'}, {'name': 'Chinn...",7,1709.02349v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",9,We present MILABOT: a deep reinforcement learn...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",A Deep Reinforcement Learning Chatbot,2017


Из всего датасета нам пригодится только столбец `summary`:

In [4]:
from random import choice

texts = arXiv_data["summary"].tolist()
print(choice(texts))

We study the problem of learning with label proportions in which the training
data is provided in groups and only the proportion of each class in each group
is known. We propose a new method called proportion-SVM, or $\propto$SVM, which
explicitly models the latent unknown instance labels together with the known
group label proportions in a large-margin framework. Unlike the existing works,
our approach avoids making restrictive assumptions about the data. The
$\propto$SVM model leads to a non-convex integer programming problem. In order
to solve it efficiently, we propose two algorithms: one based on simple
alternating optimization and the other based on a convex relaxation. Extensive
experiments on standard datasets show that $\propto$SVM outperforms the
state-of-the-art, especially for larger group sizes.


Пока что для обучения наши тексты не годятся, сначала нужно провести токенизацию:

In [5]:
!pip install razdel

In [6]:
from razdel import tokenize

SOS_TOKEN = '[SOS]'
EOS_TOKEN = '[EOS]'
PAD_TOKEN = '[PAD]'

vocabulary = set([SOS_TOKEN, EOS_TOKEN, PAD_TOKEN])
tokenized_texts = list()

for text in tqdm(texts[:1000]):
    # Токенизируем текст
    tokenized_text = tokenize(text.lower())
    tokenized_text = [token.text for token in tokenized_text]
    tokenized_text = [SOS_TOKEN] + tokenized_text + [EOS_TOKEN]

    # Обновим словарь
    for token in tokenized_text:
        vocabulary.add(token)

    # Добавим токенизированный текст в датасет
    tokenized_texts.append(tokenized_text)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [7]:
print(f"Vocabulary size is {len(vocabulary)}")
print(f"Tokenized example: {choice(tokenized_texts)}")

Vocabulary size is 9940
Tokenized example: ['[SOS]', 'a', 'growing', 'field', 'in', 'robotics', 'and', 'artificial', 'intelligence', '(', 'ai', ')', 'research', 'is', 'human-robot', 'collaboration', ',', 'whose', 'target', 'is', 'to', 'enable', 'effective', 'teamwork', 'between', 'humans', 'and', 'robots', '.', 'however', ',', 'in', 'many', 'situations', 'human', 'teams', 'are', 'still', 'superior', 'to', 'human-robot', 'teams', ',', 'primarily', 'because', 'human', 'teams', 'can', 'easily', 'agree', 'on', 'a', 'common', 'goal', 'with', 'language', ',', 'and', 'the', 'individual', 'members', 'observe', 'each', 'other', 'effectively', ',', 'leveraging', 'their', 'shared', 'motor', 'repertoire', 'and', 'sensorimotor', 'resources', '.', 'this', 'paper', 'shows', 'that', 'for', 'cognitive', 'robots', 'it', 'is', 'possible', ',', 'and', 'indeed', 'fruitful', ',', 'to', 'combine', 'knowledge', 'acquired', 'from', 'interacting', 'with', 'elements', 'of', 'the', 'environment', '(', 'affordance

In [8]:
id_to_token = list(vocabulary)
token_to_id = {token: id for id, token in enumerate(id_to_token)}

Всё готово, осталось разделить выборку на обучающую и валидационную. Вторая нужна нам для промежуточного отслеживания качества:

In [9]:
from torch.utils.data import Dataset, DataLoader

class TextsForLM(Dataset):
    def __init__(self, texts):
        self.texts = list()

        for text in tqdm(texts):
            text_ids = [token_to_id[token] for token in text]

            self.texts.append(text_ids)

    def __getitem__(self, index):
        return self.texts[index]

    def __len__(self):
        return len(self.texts)

In [10]:
from sklearn.model_selection import train_test_split

train_texts, val_texts = train_test_split(tokenized_texts, test_size=0.1,
                                          random_state=42)

train_dataset = TextsForLM(train_texts)
val_dataset = TextsForLM(val_texts)

  0%|          | 0/900 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [17]:
def collate_texts(batch):
    max_length = 0
    for text_ids in batch:
        max_length = max(max_length, len(text_ids))

    for i in range(len(batch)):
        batch[i] += [token_to_id[PAD_TOKEN]] * (max_length - len(batch[i]))

    return torch.LongTensor(batch)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True,
                              collate_fn=collate_texts)

val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False,
                              collate_fn=collate_texts)

## Реализация модели

Данные готовы, пора приступить к реализации модели

In [11]:
!pip install pytorch_lightning

     |████████████████████████████████| 527 kB 5.0 MB/s 
     |████████████████████████████████| 134 kB 48.3 MB/s 
     |████████████████████████████████| 596 kB 28.9 MB/s 
     |████████████████████████████████| 829 kB 36.8 MB/s 
     |████████████████████████████████| 397 kB 51.0 MB/s 
     |████████████████████████████████| 952 kB 39.0 MB/s 
     |████████████████████████████████| 1.1 MB 38.8 MB/s 
     |████████████████████████████████| 271 kB 49.4 MB/s 
     |████████████████████████████████| 94 kB 2.8 MB/s 
     |████████████████████████████████| 144 kB 50.5 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=5f713f5bbb65af4ac01a8933ba34dcd88e8aa74ae67d26af090e2616ddae212d
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successful

In [12]:
from pytorch_lightning import LightningModule

In [14]:
from torch import nn, optim

class LMModel(LightningModule):
    def __init__(self, vocab_size, emb_dim=128, lstm_hidden_dim=128,
                 lstm_num_layers=2):
        super().__init__()

        self.embedding_layer = nn.Embedding(vocab_size, emb_dim)
        
        self.lstm = nn.LSTM(input_size=emb_dim, hidden_size=lstm_hidden_dim,
                            batch_first=True, num_layers=lstm_num_layers)

        self.output_layer = nn.Linear(lstm_hidden_dim, vocab_size)

    def forward(self, input_ids):
        # input_ids: [batch_size, seq_len]

        # embeddings: [batch_size, seq_len, emb_dim]
        embeddings = self.embedding_layer(input_ids)

        # context_repr: [batch_size, seq_len, lstm_hidden_dim]
        # h_n: [num_layers, batch_size, lstm_hidden_dim]
        # c_n: [num_layers, batch_size, lstm_hidden_dim]
        context_repr, (h_n, c_n) = self.lstm(embeddings)

        # logits: [batch_size, seq_len, vocab_size]
        logits = self.output_layer(context_repr)

        return logits, (h_n, c_n)

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=3e-3)

        return optimizer

    def training_step(self, batch, _):
        #  input: SOS I      love   cats EOS
        # target: I.  love   cats.  EOS

        # pred:   [0.25, 0.25, 0.3, 0.0, 0.2]
        # target: [0.    0.    1.   0.   0. ]

        logits, (h_n, c_n) = self.forward(batch)

        batch_size, seq_len, vocab_size = logits.shape

        pred = logits[:, :-1, :].reshape(batch_size * (seq_len - 1), vocab_size)
        target = batch[:, 1:].reshape(batch_size * (seq_len - 1))

        loss_fn = nn.CrossEntropyLoss(ignore_index=token_to_id[PAD_TOKEN])

        loss = loss_fn(pred, target)

        return loss

In [15]:
lm_model = LMModel(len(vocabulary))

In [18]:
sample = next(iter(train_dataloader))

print(f"Sample shape is {sample.shape}")
print(f"Sample: {sample}")

Sample shape is torch.Size([16, 320])
Sample: tensor([[9239, 8186, 3754,  ..., 9878, 9878, 9878],
        [9239, 7756, 5914,  ..., 9878, 9878, 9878],
        [9239, 8186, 2954,  ..., 9878, 9878, 9878],
        ...,
        [9239, 7756, 6691,  ..., 9878, 9878, 9878],
        [9239, 5299, 1712,  ..., 9878, 9878, 9878],
        [9239, 7756, 3429,  ..., 9878, 9878, 9878]])


## Обучение модели

Без лишних слов приступим к обучению модели

In [19]:
from pytorch_lightning import Trainer

trainer = Trainer(gpus=1, max_epochs=10)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [20]:
trainer.fit(lm_model, train_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Missing logger folder: /content/lightning_logs

  | Name            | Type      | Params
----------------------------------------------
0 | embedding_layer | Embedding | 1.3 M 
1 | lstm            | LSTM      | 132 K 
2 | output_layer    | Linear    | 1.3 M 
----------------------------------------------
2.7 M     Trainable params
0         Non-trainable params
2.7 M     Total params
10.747    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

## Оценка качества

Нам определённо хочется понять, что мы только что наобучали

Прежде всего в нашем распоряжении есть метод пристального взгляда: нагенерим кучу текстов и будем оценивать их с точки зрения coherence и diversity. В идеале хотим получить разнообразные тексты, имеющие смысл. Однако это всё ещё не количественная характеристика, а просто наши субъективные наблюдения.

Посмотрим на используемый нами лосс:

$$CrossEntropyLoss(y_1, \dots, y_n) = - \sum_{t=1}^{n} \log p(y_t | y_{<t})$$

Введём следующую метрику:

$$Perplexity(y_1, \dots, y_n) = 2^{\frac{1}{n} CrossEntropyLoss(y_1, \dots, y_n)}$$

Наилучшим значением перплексии будет единица. Такой случай будет означать, что наша модель идеально выдаёт распределение токенов (ставит вероятность 1 нужному токену, соответственно лосс равен нулю). Конечно, на деле такого будет очень трудно добиться.

Наихудшим значением перплексии будет $V = vocab\_size$:

$$Perplexity(y_1, \dots, y_n) = 2^{\frac{1}{n} CrossEntropyLoss(y_1, \dots, y_n)} = 2^{- \frac{1}{n} \sum_{t=1}^{n} \log p(y_t | y_{<t})} = 2^{-\frac{1}{n} \cdot n \cdot \log \frac{1}{V}} = 2^{\log V} = V$$

Модель считает, что все токены равновероятны, а это, конечно же, никогда не так.

In [21]:
def compute_perplexity(model, batch):
    values = []
    for token_ids in batch:
        logits, _ = model.forward(token_ids.unsqueeze(0))

        batch_size, seq_len, vocab_size = logits.shape

        pred = logits[0, :-1, :]
        target = token_ids[1:]

        loss_fn = nn.CrossEntropyLoss(ignore_index=token_to_id[PAD_TOKEN])

        loss = loss_fn(pred, target)

        values.append(2 ** loss.item())

    return torch.mean(torch.tensor(values))

In [22]:
print(f"Вспомним размер словаря: {len(vocabulary)}")

Вспомним размер словаря: 9940


In [23]:
untrained_lm_model = LMModel(len(vocabulary))

In [24]:
untrained_perplexities = []
trained_perplexities = []

for batch in tqdm(val_dataloader):
    untrained_perplexities.append(compute_perplexity(untrained_lm_model, batch))
    trained_perplexities.append(compute_perplexity(lm_model, batch))

print(f"Untrained Perplexity: {torch.mean(torch.tensor(untrained_perplexities))}")
print(f"Trained Perplexity: {torch.mean(torch.tensor(trained_perplexities))}")

  0%|          | 0/7 [00:00<?, ?it/s]

Untrained Perplexity: 591.8797607421875
Trained Perplexity: 43.79312515258789


## Генерация

Рассмотрим несколько подходов к семплированию

In [26]:
def choose_argmax(logits):
    """Выбирает наиболее вероятный токен"""

    next_token_id = logits[0, -1].argmax(dim=-1)

    return next_token_id

def sample_from_distribution(logits):
    """Строит распределение по логитам и семплирует из него"""

    dist = torch.distributions.categorical.Categorical(logits=logits[0, -1])
    next_token_id = dist.sample().item()

    return next_token_id

def sample_top_k_from_distribution(logits, k=40):
    """Выбирает k наиболее вероятных токенов и семплирует из них"""

    topv, topi = logits[0, -1].topk(k)

    dist = torch.distributions.categorical.Categorical(logits=topv)
    next_token_id = topi[dist.sample().item()].item()

    return next_token_id

def nucleus_sampling(logits, p=0.95):
    """Выбирает минимальный набор токенов, чья суммарная вероятность не меньше p,
       а затем семплирует из этого набораъ
       
       Подробнее: https://openreview.net/pdf?id=rygGQyrFvH
    """
    
    return None

def generate_sample(model, beginning, max_length,
                    sampling_strategy=sample_from_distribution,
                    temperature=1.0):
    
    if beginning is None:
        tokens = [token_to_id[SOS_TOKEN]]
    else:
        tokens = [token_to_id[token.text] for token in tokenize(beginning.lower())]
        tokens = [token_to_id[SOS_TOKEN]] + tokens

    for _ in range(max_length):
        tokens_tensor = torch.LongTensor(tokens).unsqueeze(0)

        logits, _ = model(tokens_tensor)
        logits /= temperature

        next_token_id = sampling_strategy(logits)

        if next_token_id == token_to_id[EOS_TOKEN]:
            break

        tokens.append(next_token_id)

    generated_sample = ' '.join([id_to_token[id] for id in tokens[1:]])

    return generated_sample

In [38]:
generate_sample(lm_model, "We propose", 20, sampling_strategy=sample_top_k_from_distribution)

'we propose a novel approach based model such as supervision , we propose a novel architecture , we examine the proposed approach'

# Библиография

Если есть желание ещё лучше разобраться в теме, можно изучить следующие материалы:

1. Блогпост Андрея Карпати про CharRNN: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

2. Глава в учебнике Лены Войты: https://lena-voita.github.io/nlp_course/language_modeling.html

3. Статья про генерацию текста в произвольной последовательности: https://arxiv.org/pdf/2102.11008.pdf

4. Nucleus Sampling: https://openreview.net/pdf?id=rygGQyrFvH